In [1]:
import math
import random
import time
import copy
import datetime as dt
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Kalah Rules

### Game structure:
<p>The playing field consists of six houses per side and additionally one large house per player at the edge, the so-called storage. At the beginning of the game, four seeds are placed in each house except the storage.</p>

![title](images/Board.png)

### Gameplay:
<p>Alternately, a player selects a house on his side of the board. The player then places the seeds in the following houses in a counter-clockwise direction. If visited, a seed is placed in the own storage, the opponent's storage is left out. The player's turn ends if all seeds are placed.

There are some special rules, that apply if the last seed is placed in the player's storage or an empty player's house:
1. If the last seed is placed in the own storage, it is the player's turn again. 
2. If the last seed is placed in an empty house on the player's own side of the board, this seed and all the seeds in the opposite house are placed in the player's storage. This can only occur if the opposite house contains atleast one seed.</p>

### End of the game:
<p>The game ends when all the houses of one player have been emptied. The opposing player then places all remaining seed in his storage.</p>

### Object of the game:
<p>The winner is the player who has more seeds in his storage at the end of the game.</p>

Source: http://gambiter.com/mancala/Kalah.html (09.03.2022), http://www.iggamecenter.com/info/en/kalah.html (09.03.2022)

# Kalah game definition

### Basic definition:
<p>We define the game G as a six-tuple as follows so that a computer is able to play Kalah.</p>
<br>
<center><em>G = &ltStates, s0, Players, nextStates, finished, utility></em></center>

The components have the following meanings:

1. **States** is a set that contains all possible states of the game Kalah. A state of the game is represented by a list containing two lists, representing the houses of the two players. The first six values of each list represent the number of seeds in the player's houses represented by the letters **{A, B, C, D, E, F}**. The seventh and last value stands for the number of seeds in the corresponding player's storage. The start state of the game is for example:

<center><em>[[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]]</em></center>

2. **s0 $\in$ States** is the start state.

3. **Players** is a list that contains the players. Kalah is a game for exactly two players, therefore this game's **Players** list only contains two elements.
4. **nextStates** is a function which calculates a set of states that are reachable by one move from Player p in the state s. To do so, the function receives a state **s $\in$ States** and a player **p $\in$ Players**. The formula is given as follows:

<center><em>nextStates: States $\times$ Players &rarr; 2<sup>States</sup></em></center>

5. **finished** is a function that takes a state **s $\in$ States** and checks if the game is finished, meaning that one of the players has emptied all their houses. The formula is: 
<br><br>
<center><em>finished: States &rarr; $\mathbb{B}$</em></center>
<br>
The function finished is used to compute a set TerminalStates that contains all states from a finished game. This set is defined as follows: 
<br><br>
<center><em>TerminalStates:= {s $\in$ States | finished(s)}</em></center>
<br>
6. **utility** is a function that calculates the value of the game for a player. Therefore, the function takes a state **s $\in$ TerminalStates** and a player **p $\in$ Players**. The value that the function returns is an element from the set **{-1, 0, 1}**. The player p has lost the game when the function returns -1. If the function returns 1, the player has won the game and if the value is 0, the game ends in a draw. The formula for the function is:

<center><em>utility: TerminalStates $\times$ Players &rarr; {-1, 0. 1}</em></center>

source: https://github.com/karlstroetmann/Artificial-Intelligence/blob/master/Lecture-Notes/artificial-intelligence.pdf, S.87, Abruf am 06.02.2022

# Project Structure

<b style="background-color:yellow; color:blue">TODO:</b>

Describe that there are global variables, global functions and classes.
Why is the project structured this way?

## Global Variables

Additionally to the classes, there are three global variables:
- *gOrder*
- *gSeedStartNumber*
- *gStartState*

The global variable *gOrder* has the purpose to help converting the state of the game board, which is represented by a nested list, to letters. The letter representation is mainly for the UI, but also distributes to print and logging messages being easier to humanly comprehend.

The global variable *gSeedStartNumber* defines the number of seeds in every house at the start of the game. By the use of this variable it is possible to play the game in different difficulties. The standard value is four seeds per house.

The global variable *gStartState* generates the start state of the game from the variable *gSeedStartNumber*. As explained above, this results in the start state for the value four being:
<em>[[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]]</em>


In [2]:
gSeedStartNumber = 4

In [3]:
gStartState = [[gSeedStartNumber for i in range(6)]+[0] for i in range(2)]
gStartState

[[4, 4, 4, 4, 4, 4, 0], [4, 4, 4, 4, 4, 4, 0]]

In [4]:
gOrder = [['A','B','C','D','E','F','O'],
         ['a','b','c','d','e','f','o']]

## Global Game Functions

- *other_player(player_num)*
- *move(state, player_num, choice)*
- *next_states(state, player_num)*
- *finished(state)*
- *utility(state, player_num)*
- *value(state, player_num)*

### Function: other_player
The function *other_player(player_num)* returns the opponent's player number to a given player number. This means that it returns 1 for the input 0, and 0 for the input 1.

In [5]:
def other_player(player_num):
    return (player_num + 1) % 2

### Function move
The function *move* calculates the resulting state after a player choses a house in their turn and if they get another move.
Additionally, the function is used in *next_states* to calculate all possible following states from one state.

##### The calculations are implemented based on the following game rules:

1. The current player choses one of his houses. The seeds from the chosen house are placed counterclockwise in the houses of both players and in the store of the current player. The store of the opponent is left out. Then it is the turn of the opponent.

2. If the last seed is placed in the store of the current player, they get another move.

3. If the last seed is placed in an empty house of the current player the seed from this house and all seeds from the opposite house are place in the store of the current player. Then it's the opponent's turn.

To implement the function *move*, there are several auxiliary function used:
- *place_seeds*
- *check_another_turn*
- *steal_if_possible*
<br>

#### Auxiliary Function: place_seeds
The auxiliary function *place_seeds* is used in the function *move* to place the seeds from the chosen house according to the first game rule.

To do so, the function takes the current game state, the seed number of the chosen player house and the number of that house (choice). While placing the seeds, the state of the game is updated with every seed. In the end, the end state after placing all seeds is returned together with the player and house numbers of the last house in which a seed has been placed.

In [6]:
def place_seeds(state, seeds, player_num, choice):
    placing_house_num = choice
    placing_player_num = player_num
    # Go through houses counterclockwise
    while seeds > 0:
        placing_house_num += 1
        # Skip opponent's store
        if placing_house_num == 6 and placing_player_num != player_num:
            continue
        # Switch to houses of the other player
        if(placing_house_num > 6):
            placing_house_num = 0
            placing_player_num = other_player(placing_player_num)
        # Add seed to the currently visited house
        state[placing_player_num][placing_house_num] += 1
        seeds -= 1
    return state, placing_player_num, placing_house_num

#### Auxiliary Function: check_another_turn

The auxiliary function *check_another_turn* checks after all seeds where placed if the current player should get another turn according to the second game rule. This is the case whenever the last seed of the player was placed in their own store.

The function receives the resulting game state after *place_seeds* was executed as well as the player and house numbers of the last house in which a seed has been placed. It returns True if the current player gets another turn and False otherwise.


In [7]:
def check_another_turn(state, player_num, last_player_num, last_house_num):
    return last_player_num == player_num and last_house_num == 6 and not finished(state)

#### Auxiliary Function: steal_if_possible
The auxiliary function *steal_if_possible* checks after all seeds where placed if the current player can steal any seeds from their opponent according to the third game rule. This is the case whenever the last seed of the player was placed in an own empty house and the opponent has any seeds in the opposite house. In this case the own last seed together with the opposite seeds of the opponent are placed in the current player's store.

The function receives the resulting game state after place_seeds was executed as well as the player and house numbers of the last house in which a seed has been placed. *steal_if_possible* checks if any seeds can be stolen and updates the game state in that case. In the end, the possibly updated game state is returned.

In [8]:
def steal_if_possible(state, player_num, last_player_num, last_house_num):
    state = copy.deepcopy(state)
    if last_house_num == 6 or player_num != last_player_num:
        return state
    last_house_seeds = state[player_num][last_house_num]
    other_player_num = other_player(player_num)
    opponent_house_num = 5-last_house_num
    last_house_opponent_seeds = state[other_player_num][opponent_house_num]
    if last_house_seeds == 1 and last_house_opponent_seeds != 0:                
        # Collect all seeds to be rewarded and empty both houses
        received_seeds = last_house_seeds + last_house_opponent_seeds
        state[other_player_num][opponent_house_num] = 0
        state[player_num][last_house_num] = 0
        # Award all the seeds to the current player's store
        state[player_num][6] += received_seeds
    return state

#### Function move:

The function move uses all auxiliary functions described above to calculate the resulting game state from the current player's house choice. Therefore it is given the current game state, the number of the player which performs the move and the house number which that player has chosen.

At first, the function creates a copy of the state so that no changes are directly applied to the state variable in the game class. Next, the number of seeds in the chosen house are saved and the house is emptied. Afterwards, *place_seeds* is used to place the seeds counterclockwise in the houses and *steal_if_possible* is called to perform any steals afterwards if required. In the end, *another_turn* calculates if the current player should receive another turn and this information is returned from the function *move* together with the resulting new game state.

In [9]:
def move(state, player_num, choice):
    if choice not in range(6):
        raise ValueError("The choice must be 0, 1, 2, 3, 4 or 5!")
    new_state = copy.deepcopy(state)
    seeds = new_state[player_num][choice]
    new_state[player_num][choice] = 0
    # Place all seeds and check for special rules
    new_state, last_player_num, last_house_num = place_seeds(new_state, seeds, player_num, choice)
    new_state = steal_if_possible(new_state, player_num, last_player_num, last_house_num)
    another_turn = check_another_turn(new_state, player_num, last_player_num, last_house_num)
    return new_state, another_turn

### Function: next_states

The function  next_states computes all reachable states resulting from the current player and state. that player has chosen. Therefore it is given the current game state and the number of the player who is doing the next move.

To find the reachable states the function executes a move for each possible house (that contains at least one seed). If the player gets another turn the next_states function is executed again for the resulting state until the player has finished. Then all the new states are saved in a list and returned.  

In [10]:
def next_states(state, player_num):
    states = []
    for choice in range(6):
        # Check if choice is valid (at least one seed in house)
        if state[player_num][choice] == 0:
            continue   
        next_state, another_turn = move(state, player_num, choice)

        # Check if player has another turn
        # If so, next_states is called recursively until the player has no other turn 
        if another_turn:
            for s,choices in next_states(next_state, player_num):
                states.append((s,[choice] + choices))
        else:
            states.append((next_state,[choice]))

    return states

### Function: finished
The private function *finished()* checks if one of the players has no seeds in their house left and is therefore unable to take another turn. If this is the case, the function returns True, otherwise it returns False. 

In [11]:
def finished(state):
    for side in state:
        fin = True
        for house in side[:-1]:
            if house != 0:
                fin = False
        if fin:
            return True
    return False

### Function: utility
The  function *utility(player_num)* receives the number of the current player and uses the current state to calculate the utility of the state for that player. The function compares the seeds from both stores and returns an Integer dependent on the result. If the current player has more seeds than the opponent the function returns 1. If they own less seeds than the opponent the function returns -1 and if it is a draw the number 0 is returned.     

In [40]:
def heuristic(state, player_num):
    playerStore = state[player_num][-1]
    opponentStore = state[other_player(player_num)][-1]
    totalSeedNum = 6*2*gSeedStartNumber
    return min((playerStore-opponentStore)/(totalSeedNum/2+1), 1)

In [13]:
def utility(state, player_num):
    if not finished(state):
        return heuristic(state, player_num)
    playerStore = sum(state[player_num])
    opponentStore = sum(state[other_player(player_num)])
    if(playerStore > opponentStore):
        return 1
    elif(playerStore == opponentStore):
        return 0
    else:
        return -1

### Function: value


#### Auxiliary functions for the method value

The method *to_tuple* takes a list of the two lists that represent a game state. The function converts the list to a tuple of tuples and returns it.

The method *to_list* does that procedure vice versa.

In [14]:
def to_tuple(state_list):
    return tuple(tuple(s) for s in state_list)

In [15]:
def to_list(state_tuple):
    return list(list(s) for s in state_tuple)

#### Function: memoize

The function *memoize* computes a memoized version of a function f that is given as parameter. At first a dictionary named Cache is created that is used as a memory cache for the function *memoized*. At first *memoized* tries to retrieve the value of the given function f from the dictionary Cache and returns the value. If *memoize* can't retrieve a value the function f is called to compute the result and the result is stored in the Cache.   

In [16]:
Cache = {}

def memoize(f):
    global Cache

    def f_memoized(*args):
        args = (to_tuple(args[0]),args[1],args[2])
        if args in Cache:
            return Cache[args]
        
        result = f(to_list(args[0]),args[1],args[2])
        Cache[args] = result
        return result

    return f_memoized

The function *value* takes a *state* and a *player*. In addition, a *limit* is given to define the recursion depth, which limits the number of next states which are calculated and evaluated. Similar to the function *utility*, the function *value* returns a value from the set {-1, 0, +1}. The base case of the recursive function *value* is the case that the game is finished or if the limit has reached the value 0. In these cases, the *utility* function is called. 

In the other cases, the next states are computed for both players alternately until the limit is reached or the game reaches a finished state. Because the gains of the opponent are the losses of player p, the negative output of *value(n, o)* is taken when calculating the current game value to the opponent.   

In [17]:
@memoize
def value(state, player_num, limit):
    if finished(state) or limit==0:
        return utility(state, player_num)
    other = other_player(player_num)
    return max([-value(ns, other, limit-1) for ns,_ in next_states(state, player_num)])

<b style="background-color:yellow; color:blue">Comments:</b> 
* This is not good enough without the use of a heuristic.  If `limit == 0`, then instead of
  returning the `utility`, a value computed by a heuristic should be computed.
* You should analyze how much the program speeds up because of memoization.
* Logistische Funktion von Verhältnis - 1.
* limit -=1 muss ans Ende, oder?

# Player Class

#### Attributes:
- *player_type*
- *number*
- *name*

#### Methods:

- *\_\_init__(number, name)*
- *\_\_str__()*
- *_available_house(own_house)*
- *choose_house(current_state)*

The class <tt>Player</tt> is the superclass which represents a Kalah game player. In Kalah there are exactly two players which compete against each other. 

The attribute *player_type* is only for subclasses of the <tt>Player</tt> class. It contains a string representation of the name of the <tt>Player</tt> subclass. This makes it possible to print the specific type of a <tt>Player</tt> object. This is for example used for creating log files. 

The attribute *number* of a player has either the value 0 or 1 depending on the order in which the players take their turns.
If the number of the player is 0, they start the game by having the first turn.
Additionally, *number* represents the index of the state list which contains the player's list of house values.

The attribute *name* is a string which represents the player's name in the UI or print and logging messages.

### Method: __init__
The method *\_\_init__(number, name)* initializes the <tt>Player</tt> object and checks if the given *number* is either 0 or 1 and if the given *name* is a string. If one of these criteria is not met, an error message is raised.

### Method: __str__
The method *\_\_str__()* defines the *name* of the <tt>Player</tt> object as their string representative for print messages.

In [18]:
class Player:
    
    def __init__(self, number, name):
        
        if number in [0,1]:
            self.number = number
        else:
            raise ValueError("Number of player must be 0 or 1!")
        if isinstance(name, str):
            self.name = name
        else:
            raise ValueError("Name must be a string!")
            
    def __str__(self):
        return self.name

### Method: _available_houses

The private method *_available_houses(own_houses)* receives a list which represents the player's houses and their store. The method returns a list with the indices of all houses which contain at least one seed (list indices with a value higher than null).

In [19]:
def _available_houses(self, own_houses):
    return [i for i, n in enumerate(own_houses) if n != 0]

Player._available_houses = _available_houses

### Method: choose_house

The method *choose_house(current_state)* receives the current state of the Kalah board and returns the index of the chosen house from the player's house list. This method is used to distinguish the player subclasses and is therefore implemented differently in each of them. There is no basic implementation of this method in the <tt>Player</tt> class. This is why in the class <tt>Kalah_Game</tt> only subclasses of <tt>Player</tt> are accepted as players. Therefore, this class is intended as an abstract class.

In [20]:
def choose_house(self, current_state):
    pass

Player.choose_house = choose_house

## Human Player Class

The class <tt>Human</tt> inherits from <tt>Player</tt>.
It is used for humans playing against each other or against one of the AIs. 

### Method: __init__
The method *\_\_init__(number, name)* initializes the <tt>Human Player</tt> object and checks if the given *number* is either 0 or 1 and if the given *name* is a string. If one of these criteria is not met, an error message is raised. Additionally the variable player_type is set to identify the Object.


In [21]:
class Human(Player):
    
    def __init__(self, number, name):
        self.player_type = "Human"
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* for the <tt>Human</tt> class is implemented as follows:

At first the own houses are extracted from the game state and the available house indices are calculated using the method *_available_houses*. Afterwards the human player is asked to choose one of the available houses via an input field. If the player enters an invalid value, they are asked to input a value again until a valid value is given. The index of the corresponding house is returned.

In [22]:
def choose_house(self, current_state):
    own_t,store = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_houses(own_t)

    i_string = "Choose one of the available houses:\n"
    for i in available:
        i_string += f"{gOrder[self.number][i]}, "

    choice_str = ""
    letter_numbers = {gOrder[1][i]:i for i in range(6)}
    
    while choice_str not in [k for k in letter_numbers if letter_numbers[k] in available]:
        choice_str = input(i_string[:-2]+"\n").lower()

    choice = letter_numbers[choice_str]
    IPython.display.clear_output()
    return choice
    
Human.choose_house = choose_house

## Repeated Player Class

The class <tt>Repeated_Player</tt> inherits from <tt>Player</tt>.
It is used for repeating the behavior of a player from a previous game on basis of a given log file.

### Method: __init__
The method *\_\_init__(number, name, logged_moves)* initializes the <tt>Repeated Player</tt> object and checks if the given *number* is either 0 or 1 and if the given *name* is a string. If one of these criteria is not met, an error message is raised. Additionally the variable player_type is set to identify the Object.

Furthermore the functions recieves a list of moves **logged_moves**, which is used to later on replay the game at a different time.

In [23]:
class Repeated_Player(Player):
    
    def __init__(self, number, name, logged_moves):
        self.player_type = "Repeated_Player"
        self.moves_to_repeat = logged_moves
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* for the <tt>Repeated_Player</tt> class is implemented as follows:

The list *moves_to_repeat* is handled as a stack in this method. This means that one after one, the moves which stand at the beginning of the list are extracted and removed from the list. In the beginning of the method this process is repeated until the current state of the game matches the state of the last removed move from the *moves_to_repeat* list. This means, that the move which the player will do next is should now be at the beginning of the list.

In the next step, this move is also extracted from the list, but not removed as it could contain the current state for the next move of the player if they gain another turn. To make sure, that this move was actually done by this player in the logged game, the logged move player number is compared to the actual player number. If the numbers match, the choice from that move is returned.

In [24]:
def choose_house(self, current_state):
    
    state = self.moves_to_repeat.pop(0)[2]
    while state != current_state:
        if len(self.moves_to_repeat) == 0:
            raise ValueError("There is no move left to repeat!")
        state = self.moves_to_repeat.pop(0)[2]
        
    this_move = self.moves_to_repeat[0]
    if this_move[0] not in (-1,self.number):
        raise ValueError("This move was originally not played by this player!")
        
    choice = this_move[1]
        
    return choice

Repeated_Player.choose_house = choose_house

## Random_AI Player Class

The class <tt>Random_AI</tt> inherits from <tt>Player</tt>. It is a simple AI player implementation which chooses the houses at random.

### Method: __init__
The method *\_\_init__(number, name,seed)* initializes the <tt>Random_AI Player</tt> object and checks if the given *number* is either 0 or 1 and if the given *name* is a string. If one of these criteria is not met, an error message is raised. Additionally the variable player_type is set to identify the Object.

Furthermore this function recieves an Integer **seed** which is used to set the seed for the random number calculations provided by the **random** library.

In [25]:
import random as rn

class Random_AI(Player):
    
    def __init__(self, number, name, seed):
        self.player_type = "Random_AI"
        self.seed = seed
        rn.seed(seed)
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* for the <tt>Random_AI</tt> class is implemented as follows:

At first, the own houses are extracted from the game state and the indices of the available houses are calculated using the method *_available_houses*. From this list, one is chosen at random using the function **choice** from the library **random**. Afterwards, this value is returned.

In [26]:
def choose_house(self, current_state):
    own_t,safe = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_houses(own_t)
    choice = rn.choice(available)

    return choice

Random_AI.choose_house = choose_house

# Minimax Player Class

The class <tt>Minimax</tt> inherits from <tt>Player</tt>. It is an AI player implementation which chooses a following state by calculating all possible next states for a defined limit depth. It then chooses the option which guarantees them a higher number of seeds in their store than their opponent. If there are several options, the choice which results in the highest number of seeds in the player's store is chosen.

The class <tt>Minimax</tt> has a modified *\_\_init__* function which takes the additional argument *limit* which is only relevant to the method *choose_house*. There the value of *limit* defines, for how many recursion steps the function *value* should be called. This means that *limit* defines how many next game states should be calculated to form the decision of the <tt>Minimax</tt> AI.

The *limit* should be chosen carefully, as higher limits increase the recursion depth linearly and the computing time exponentially. Additionally, *limit* has to be greater than 1 at all times. 

Ideally *limit* should be a number from 2 - 4 (incl.).

In [42]:
class Minimax(Player):
    
    def __init__(self, number, name, limit, seed=0):
        self.player_type = "Minimax"
        self.limit = limit
        self.seed = seed
        rn.seed(seed)
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(self,current_state)* is the core method of the <tt>Minimax</tt> algorithm. It calculates the best choices for a <tt>Minimax</tt> player, using the previously defined functions *value(state, player_num, limit)* and *next_states(state, player_num)*.

First, the best possible value from {-1, 0, 1} (meaning loss, draw or win) that can be reached with the current state of the game is calculated. Following that, all next possible states are checked and only those are elected as eligible, which reach the previously calculated best possible value.

Afterwards, the best choice is being made by comparing all eligible choices for the highest player's Kalah store value.

In [28]:
def choose_house(self, current_state):
    
    NS = next_states(current_state, self.number)
    bestVal = value(current_state, self.number,self.limit)

    BestChoices = [choices[0] for state,choices in NS if -value(state, other_player(self.number),self.limit-1) == bestVal]
    
    if len(BestChoices) == 0:
        raise ValueError(f'No choice for Minimax found! \nbestVal: {bestVal}\nBestChoices: {BestChoices}\nnext_states: {NS}')

    # Find best choice, where number of seeds in own Store is maximized (out of next_states)
    best_choice = rn.choice(BestChoices)

    return best_choice

Minimax.choose_house = choose_house

# AlphaBeta Player Class

The class <tt>AlphaBeta</tt> inherits from <tt>Player</tt>. It is an AI player implementation which chooses a following state by calculating all possible next states for a defined limit depth. It then chooses the option which guarantees them a higher number of seeds in their store than their opponent. If there are several options, the choice which results in the highest number of seeds in the player's store is chosen.

In [29]:
class AlphaBeta(Player):
    
    def __init__(self, number, name, limit):
        self.player_type = "AlphaBeta"
        self.limit = limit
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(self,current_state)* is the core method of the <tt>Minimax</tt> algorithm. It calculates the best choices for a <tt>Minimax</tt> player, using the previously defined functions *value(state, player_num, limit)* and *next_states(state, player_num)*.

First, the best possible value from {-1, 0, 1} (meaning loss, draw or win) that can be reached with the current state of the game is calculated. Following that, all next possible states are checked and only those are elected as eligible, which reach the previously calculated best possible value.

Afterwards, the best choice is being made by comparing all eligible choices for the highest player's Kalah store value.

In [30]:
def alphaBeta(State, player, alpha, beta):
    if finished(State):
        return utility(State, player)
    val = alpha
    for ns in next_states(State, player):
        val = max(val, -value(ns, other(player), -beta, -alpha))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

def value(State, player, alpha=-1, beta=1):
    global Cache
    if State in Cache:
        val, a, b = Cache[State]
        if a <= alpha and beta <= b:
            return val
        else:
            alpha = min(alpha, a)
            beta = max(beta , b)
            val = alphaBeta(State, player, alpha, beta)
            Cache[State] = val, alpha, beta
            return val
    else:
        val = alphaBeta(State, player, alpha, beta)
        Cache[State] = val, alpha, beta
        return val

In [31]:
#def heuristic(State, player):

In [32]:
def alphaBeta(State, player, limit, heuristic, alpha=-1, beta=1):
    if finished(State):
        return utility(State, player)
    if limit == 0:
        return heuristic(State, player)
    val = alpha
    for ns in next_states(State, player):
        val_ns = value(ns, other(player), limit-1, heuristic, -beta, -alpha)
        val = max(val, -val_ns)
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

In [33]:
#def choose_house(self, current_state):
#   
#  NS = next_states(current_state, self.number)
# bestVal = value(current_state, self.number,self.limit)
#
#    BestChoices = [choices[0] for state,choices in NS if -value(state, other_player(self.number),self.limit-1) == bestVal]
#    
#    if len(BestChoices) == 0:
#        raise ValueError(f'No choice for Minimax found! \nbestVal: {bestVal}\nBestChoices: {BestChoices}\nnext_states: {NS}')#
#
    # Find best choice, where number of seeds in own Store is maximized (out of next_states)
#    best_choice = rn.choice(BestChoices)
#
#    return best_choice

#Minimax.choose_house = choose_house

# Kalah_Game Class

#### Attributes:
- *state*
- *players*
- *current_player*
- *display_mode*
- *logged_moves*

#### Methods:

- *\_\_init__(players)*
- *_show_state(state)*
- *show_state()*
- *start()*
- *log_to_file(file_id)*

The class <tt>Kalah_Game</tt> is the core of the Kalah game implementation. It contains all information on the game, including the current game state.

The attribute *state* represents the state of the Kalah board which is defined by the number of seeds laying in each of the player's house and their stores. It is implemented by a nested list which contains a list for the house on each of the two players' sides. The last index of each of the lists is the store of that player. At the start of the game, where the stores are empty and there are six seeds in every house, the implemented representation of the state for example is: [[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]].

The attribute *players* is a list of the two players that play the game. They must be instances of a subclass of the <tt>Player</tt> class. The order in which they take turns is determined by their <tt>Player</tt> *number*: The player with the number 0 goes first.

The attribute *current_player* has either the value 0 or 1. It takes track of which player's turn it currently is. If *current_player* has the value 0 for example, it is the turn of the player which stands at index 0 of the *players* list and therefore also has the <tt>Player</tt>'s class attribute *number* of value 0.

The attribute *display_mode* defines which way the game is displayed in the output console. The possible values are 0, 1 and 2 and are defined as follows:
- 0: No output is displayed. The game is only logged to the log file
- 1: The board is only displayed using the print method *_show_state*
- 2: The board is displayed with the function *draw_board* using **ipycanvas** for rendering

The attribute *logged_moves* is a list containing a tuple for every move that is made in the game. This list is necessary for the method *log_to_file*. Each tuple has the player number of the player that have made the move at the first index, the index of the chosen house at the second index and the resulting game state as the third index. The start state is saved as the first move in the list, having -1 as the player number and -1 as the chosen house number. As Python uses references, the states are saved in this list by creating deep copies with the library **copy**. At the beginning of the game *logged_moves* looks like this:

\[(-1, -1, [[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]])\]

### Method: __init__
The method *\_\_init__(players)* initializes the game by setting the *state* to the initial state (seen above), initializing a board object and setting the *players* list using the received "players" argument. Before setting the *players* list, the received list is checked for the number of items it contains (must be exactly 2) and if the items in the list are both instances of a <tt>Player</tt> subclass (but not of the class <tt>Player</tt> itself). In error case, matching error messages are raised.

In [34]:
class Kalah_Game():
    
    def __init__(self, players, display_mode):
        self.state = copy.deepcopy(gStartState)
        self.turn = 0
        
        if len(players) != 2:
            raise ValueError("There must be exactly two players!")
        if not ((isinstance(players[0], Player) and type(players[0]) != Player) 
            and (isinstance(players[1], Player) and type(players[1]) != Player)):
            raise ValueError("Both players must be of instances of a subclass of the class Player!")
        if {players[0].number, players[1].number} != {0,1}:
            raise ValueError("One of the players must be self.number 0 and the other one self.number 1!")
        
        self.players = players
        self.current_player = 0
        
        if display_mode not in range(3):
            raise ValueError("The display mode must be 0, 1 or 2!")
        self.display_mode = display_mode
        
        self.logged_moves = [(-1,-1,copy.deepcopy(self.state))]

### Methods: _show_state and show_state
The private method *_show_state(state)* creates a formatted string which represents the received state and prints it to the console. It can be used as an alternative to the **ipycanvas** game UI.

The method *show_state()* calls the private method *_show_state(state)* with the current game state (attribute *state*).

In [35]:
def _show_state(self, state):
    s = f''

    s += f'{self.players[0].name}:\t\t\t'
    for j in range(6,-1,-1):
        s += f'{gOrder[0][j]}: {state[0][j]}  '
    s += f'\n'

    s += f'{self.players[1].name}:\t\t\t\t'
    for j in range(7):
        s += f'{gOrder[1][j]}: {state[1][j]}  '
    s += f'\n'

    print(s)

Kalah_Game._show_state = _show_state

In [36]:
def show_state(self):
    self._show_state(self.state)
    
Kalah_Game.show_state = show_state

### Method: start
The method *start()* starts the Kalah game. Until the game is finished (the method *_finished()* returns True), both players take turns, starting with the player with the number 0. At the start of each turn, the current game state is shown. Next, the current player chooses one of the house with the <tt>Player</tt> method *choose_house(current_state)*. Afterwards, this choice is handed to the private method *_move(player_num, choice)* which calculates the new game state. The attribute *state* is updated with this new game state. Then it is the turn of the other player. If the game is finished, the method *utility()* calculates which of the players wins the game and the result is printed to the console.

In [37]:
def start(self):
    while(not finished(self.state)):
        self.turn += 1
        if self.display_mode == 1:
            print("\nCurrent state:")
            self.show_state()
            print(f"Next is {self.players[self.current_player].name}'s turn.")
        elif self.display_mode == 2:
            print("Current state:")
            draw_board(self.state,self.turn)
            print(f"Next is {self.players[self.current_player].name}'s turn.")
            time.sleep(0.05)
        
        choice = self.players[self.current_player].choose_house(self.state)
        
        if self.display_mode != 0:
            print(f"{self.players[self.current_player].name} chose {gOrder[self.current_player][choice]}")

        self.state, another_turn = move(self.state, self.current_player, choice)
        # create move log
        move_log = tuple([self.current_player, choice, copy.deepcopy(self.state)])
        self.logged_moves.append(move_log)
        
        if not another_turn:
            self.current_player = other_player(self.current_player)
        elif another_turn and self.display_mode != 0:
            print(f"{self.players[self.current_player].name} gets another turn!")

    won0 = utility(self.state,0)
    
    if self.display_mode != 0:
        if self.display_mode == 1:
            print("")
            self.show_state()
        elif self.display_mode == 2:
            draw_board(self.state,self.turn)
            
        print("Finished Game!")  
        
        if(won0 == 1):
            print(f"{self.players[0]} wins!")
        elif(won0 == -1):
            print(f"{self.players[1]} wins!")
        else:
            print(f"Draw!")
    
Kalah_Game.start = start

### Method: log_to_file
The method *log_to_file* creates a json file from all important game information, including the player subclass types and names as well as the different moves from the game and the winner(s). With the use of this method, played games can be saved and analyzed. The created log file is saved to the folder "logs" with the given file_id in the file name.

In [38]:
import json

def log_to_file(self, file_id):
    json_dict = {
        "players":{
            0:{},
            1:{}
        },
        "moves":self.logged_moves,
        "winner":[i for i in range(2) if utility(self.state,i) != -1]
    }
    
    for i in range(2):
        json_dict["players"][i]["type"] = self.players[i].player_type
        json_dict["players"][i]["name"] = self.players[i].name
        if isinstance(self.players[i], Random_AI):
            json_dict["players"][i]["seed"] = self.players[i].seed
        if isinstance(self.players[i], Minimax):
            json_dict["players"][i]["limit"] = self.players[i].limit
    
    with open(f"logs/log_{file_id}.json", "w") as f:
        f.write(json.dumps(json_dict, indent=4))
        f.close()

Kalah_Game.log_to_file = log_to_file

<b style="background-color:yellow; color:blue">Comments:</b>
I would prefer if you would create plain text files instead of *Json* files.

## Repeat Game from Log File

The function *repeat_game* takes the filename of a log file and the display mode with which the game should be repeated as the input values. It also receives the information, if included AIs should calculate their choices new or if they should just repeat the moves that are presented in the log file. The decicions of human players are always repeated based on the log file. 

The function detects the player types and names from the file and creates players based on this information. The player types are only relevant for AIs if **repeat_AIs** is set to False. In all other cases, the players are created from the <tt>Repeated_Player</tt> class.

In the end, the game is initialized with the created players and the given display mode and then started.

In [39]:
def repeat_game(filename, repeat_AIs, display_mode):
    json_dict = {}
    
    with open("logs/"+filename) as f:
        json_dict = json.load(f)
    
    players = []
    for i in range(2):
        i = str(i)
        p_type = json_dict["players"][i]["type"]
        
        if repeat_AIs:
            players.append(Repeated_Player(int(i),json_dict["players"][i]["name"],json_dict["moves"]))
        else:
            if p_type == "Human":
                players.append(Repeated_Player(int(i),json_dict["players"][i]["name"],json_dict["moves"]))
            elif p_type == "Random_AI":
                players.append(Random_AI(int(i),json_dict["players"][i]["name"],json_dict["players"][i]["seed"]))
            elif p_type == "Minimax":
                players.append(Minimax(int(i),json_dict["players"][i]["name"],json_dict["players"][i]["limit"]))
            else:
                raise ValueError("The given player type is unknown!")
    
    game = Kalah_Game([players[0],players[1]],display_mode)
    game.start()

## <b style="background-color:yellow; color:blue">Concluding Remarks</b> 

* Testing should be more systematic, i.e. you should structure the code into functions and you should also discuss the results.
* You still need to implement *Alpha-Beta-Pruning* and *[Scout](https://www.aaai.org/Papers/AAAI/1980/AAAI80-041.pdf)*. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b91c3ea7-d814-439b-837a-72fdc90697b1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>